In [1]:
%cd /gxfs_work/cau/sunms534/trading_bot/

/gxfs_work/cau/sunms534/trading_bot


/gxfs_work/cau/sunms534/.conda/envs/my_pytorch_env/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [9]:
from pathlib import Path

import pandas as pd
from tokenizers import ByteLevelBPETokenizer
from transformers import AutoTokenizer

from tokenizers.processors import RobertaProcessing
from src.config import config
import torch

In [33]:
bodies = pd.read_parquet(config.data.news.stripped, columns=["parsed_body"])

In [32]:
# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer(lowercase=True)

tokenizer.train_from_iterator(bodies.parsed_body.values, 
                              vocab_size=30000, 
                              min_frequency=10, 
                              special_tokens=[
                                "<s>",
                                "<pad>",
                                "</s>",
                                "<unk>",
                                "<mask>",
                              ])

In [34]:
tokenizer.post_processor = RobertaProcessing(
    cls=("<s>", tokenizer.token_to_id("<s>")),
    sep=("</s>", tokenizer.token_to_id("</s>")),
)
tokenizer.enable_truncation(max_length=256)

In [37]:
# Save files to disk
tokenizer = PreTrainedTokenizerFast(tokenizer_object=tokenizer._tokenizer, pad_token="<pad>", truncation=True)
tokenizer.save_pretrained("data/models/newstokenizer")

('data/models/newstokenizer/tokenizer_config.json',
 'data/models/newstokenizer/special_tokens_map.json',
 'data/models/newstokenizer/tokenizer.json')

In [34]:
tokenizer = AutoTokenizer.from_pretrained("data/models/newstokenizer", max_len=256)

In [35]:
text = bodies.iloc[10].parsed_body

In [36]:
encoding = tokenizer(
    text, 
    add_special_tokens = True, 
    truncation = True, 
    padding = "max_length", 
    max_length = 256,
    return_attention_mask = True, 
    return_tensors = "pt"
)

In [63]:
encoding["input_ids"]

AttributeError: 'Tensor' object has no attribute 'tokens'

In [38]:
encoding["attention_mask"]

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [4]:
title_inputs_ids = pd.read_parquet(config.data.news.input_ids)
masks = pd.read_parquet(config.data.news.masks)

In [67]:
from transformers import RobertaConfig, RobertaModel, RobertaForMaskedLM, RobertaTokenizerFast
from transformers import DataCollatorForLanguageModeling

configuration = RobertaConfig(vocab_size = 30000,
                              hidden_size = 256,
                              num_hidden_layers = 6,
                              num_attention_heads = 4,
                              intermediate_size = 1556,
                              hidden_act = 'gelu',
                              hidden_dropout_prob = 0.1,
                              attention_probs_dropout_prob = 0.1,
                              max_position_embeddings = 258,
                              type_vocab_size = 2,
                              initializer_range = 0.02,
                              layer_norm_eps = 1e-12,
                              pad_token_id = 1,
                              bos_token_id = 0,
                              eos_token_id = 2,
                              position_embedding_type = 'absolute',
                              use_cache = True,
                              classifier_dropout = None)


model = RobertaModel(configuration)

In [68]:
title_inputs_ids.iloc[0, :]

0          0
1      14924
2        262
3       1411
4        293
       ...  
251      469
252     1831
253      629
254     3585
255        2
Name: 0, Length: 256, dtype: int64

In [69]:
title_inputs_ids.iloc[1:5, :].values

array([[   0,   91, 5641, ..., 2529,  267,    2],
       [   0,  267,  321, ..., 4712,  335,    2],
       [   0,  583,  302, ..., 7526,  290,    2],
       [   0,   69, 6811, ...,  716,  353,    2]])

In [70]:
model(input_ids=torch.tensor(title_inputs_ids.iloc[1:5, :].values), attention_mask=torch.tensor(masks.iloc[1:5, :].values))

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.2394, -2.3523,  0.0701,  ...,  1.2033, -0.5012, -0.6043],
         [ 0.4904, -0.6112,  0.6721,  ..., -0.0792,  0.1891,  0.0141],
         [ 0.2500, -1.8871, -0.2477,  ...,  0.4277,  0.1697,  0.6306],
         ...,
         [ 1.0402,  0.0129,  0.4128,  ..., -0.3565,  0.5749, -0.9676],
         [-0.7853, -0.2002,  0.4063,  ..., -1.2100,  0.4986, -1.1683],
         [ 1.0320, -0.8628,  0.8338,  ...,  0.7949,  1.2175, -0.4417]],

        [[ 1.4046, -2.3721,  0.3859,  ...,  1.4811, -0.1087, -1.2942],
         [-1.5018, -0.3980,  0.1484,  ...,  0.5272, -0.5977, -1.1767],
         [ 0.5308, -1.5011,  0.9497,  ...,  0.8222,  1.5338, -0.5177],
         ...,
         [ 1.8587, -0.0864, -2.1232,  ...,  0.2590, -0.3208, -1.0196],
         [ 0.9724, -0.8159,  0.9147,  ..., -0.7498,  0.2683, -0.8722],
         [ 1.0463, -0.2195,  0.7432,  ...,  0.6326,  1.1605, -0.5176]],

        [[ 1.4023, -2.3850,  0.1718,  ...,  1.2643, -